In [1]:
import gradio as gr
from transformers import MarianTokenizer, MarianMTModel
import torch


In [2]:

# Available translation models
MODEL_OPTIONS = {
    "English → German": "./marian-finetuned-en-de",
    "English → Romanian": "./finetuned-en-ro"
}


In [3]:

def load_model_and_tokenizer(model_path):
    tokenizer = MarianTokenizer.from_pretrained(model_path)
    model = MarianMTModel.from_pretrained(model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    return model, tokenizer, device


In [4]:

# Load default model at startup
active_model, active_tokenizer, active_device = load_model_and_tokenizer(MODEL_OPTIONS["English → German"])


/opt/anaconda3/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:

# Translation function with dynamic model switching
def translate_text(text, model_choice):
    global active_model, active_tokenizer, active_device

    model_path = MODEL_OPTIONS[model_choice]
    active_model, active_tokenizer, active_device = load_model_and_tokenizer(model_path)

    if not text.strip():
        return "Please enter some English text."

    inputs = active_tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(active_device)
    translated_tokens = active_model.generate(**inputs, max_length=128)
    translated_text = active_tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text


In [6]:

# Gradio Interface
dropdown = gr.Dropdown(label="Choose Translation Model", choices=list(MODEL_OPTIONS.keys()), value="English → German")
textbox = gr.Textbox(lines=3, label="Enter English Text")
outputbox = gr.Textbox(label="Translated Output")

iface = gr.Interface(
    fn=translate_text,
    inputs=[textbox, dropdown],
    outputs=outputbox,
    title="English Translator with Model Selector",
    description="Select a model (EN→DE or EN→RO), enter English text, and get your translation."
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/opt/anaconda3/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
